<a href="https://colab.research.google.com/github/HillaryKoros/geeStuff/blob/main/Supervised_Cassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install geemap

In [4]:
import ee
import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import geopandas as gpd

In [17]:
from google.colab import output
output.enable_custom_widget_manager()

In [6]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [7]:
# // Import country boundaries feature collection.
dataset = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017');

# // Apply filter where country name equals Uganda.
kenyaBorder = dataset.filter(ee.Filter.eq('country_na', 'Kenya'));

# // Add Uganda outline to the Map as a layer.
Map.centerObject(kenyaBorder, 6);
Map.addLayer(kenyaBorder);
Map

Map(bottom=4539.0, center=[0.5578063517900957, 37.84559934447033], controls=(WidgetControl(options=['position'…

In [18]:
point = ee.Geometry.Point([36.22, 1.099])
image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2021-01-01', '2021-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

Map(bottom=16604.0, center=[1.099, 36.22], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [14]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2016-03-28'

In [15]:
image.get('CLOUD_COVER').getInfo()

0.68

In [19]:
ESA = ee.ImageCollection('ESA/WorldCover/v200').first().select('Map').clip(image.geometry())
Map.addLayer(ESA, {}, 'ESA_WorldCover_v200')
Map

Map(bottom=32868.0, center=[1.098565496040652, 36.21917724609376], controls=(WidgetControl(options=['position'…

In [20]:
# Make the training dataset.
points = ESA.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, 'training', False)

In [21]:
print(points.size().getInfo())

5000


In [22]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [36.26892546936601, 1.0116626138381664]}, 'id': '0', 'properties': {'Map': 20}}


In [40]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'Map'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)
# Adds a column of deterministic pseudorandom numbers.
sample = training.randomColumn()

split = 0.8

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [41]:
training.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 295,
  'B2': 386,
  'B3': 604,
  'B4': 854,
  'B5': 1797,
  'B6': 2006,
  'B7': 1409,
  'Map': 20,
  'random': 0.18615681166021825}}

In [42]:
validation.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '4_0',
 'properties': {'B1': 204,
  'B2': 294,
  'B3': 606,
  'B4': 765,
  'B5': 2470,
  'B6': 2182,
  'B7': 1398,
  'Map': 20,
  'random': 0.9336151313849833}}

In [43]:
# Train a CART classifier with default parameters.
trained = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [44]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

Map(bottom=16688.0, center=[-0.04394530819134536, 38.35876464843751], controls=(WidgetControl(options=['positi…

In [45]:
class_values = ESA.get('Map_class_values').getInfo()
class_values

[10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]

In [46]:
class_palette = ESA.get('Map_class_palette').getInfo()
class_palette

['006400',
 'ffbb22',
 'ffff4c',
 'f096ff',
 'fa0000',
 'b4b4b4',
 'f0f0f0',
 '0064c8',
 '0096a0',
 '00cf75',
 'fae6a0']

In [47]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [48]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(bottom=16688.0, center=[-0.04394530819134536, 38.35876464843751], controls=(WidgetControl(options=['positi…

In [33]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [49]:
Map.add_legend(builtin_legend='ESA_WorldCover')
Map

Map(bottom=16688.0, center=[-0.04394530819134536, 38.35876464843751], controls=(WidgetControl(options=['positi…

In [36]:
counties = ee.FeatureCollection("projects/ee-korosprojects/assets/KEcounties")
Map.addLayer(counties, {}, 'KE counties')
Map

Map(bottom=32868.0, center=[1.098565496040652, 36.21917724609376], controls=(WidgetControl(options=['position'…

In [37]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
ESA_stats = os.path.join(out_dir, 'ESA_stats.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# statistics_type can be either 'SUM' or 'PERCENTAGE'
# denominator can be used to convert square meters to other areal units, such as square kilometers
geemap.zonal_statistics_by_group(
    landcover,
    counties,
    ESA_stats,
    statistics_type='SUM',
    denominator=1000000,
    decimal_places=2,
)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/ESA_stats.csv


In [39]:
geemap.create_download_link(ESA_stats)